# Using MedGemma as Remotely Sourced Zoo Model

In [1]:
import fiftyone as fo

from fiftyone.utils.huggingface import load_from_hub

dataset = load_from_hub(
    "Voxel51/SLAKE",
    name="SLAKE",
    overwrite=True,
    max_samples=50
    )

Loading dataset
Importing samples...
 100% |███████████████████| 50/50 [4.5ms elapsed, 0s remaining, 11.1K samples/s]      


# Setup Zoo Model

In [ ]:
import fiftyone.zoo as foz

foz.register_zoo_model_source("https://github.com/harpreetsahota204/medgemma", overwrite=True)

foz.download_zoo_model(
    "https://github.com/harpreetsahota204/medgemma",
    model_name="google/medgemma-4b-it", 
)

model = foz.load_zoo_model(
    "google/medgemma-4b-it",
    quantized=True,
    # install_requirements=True #run this to install requirements if they're not already
    )

# Use MedGemma for Classification

You can use this model for a zero-shot classification task as follows, which will add a [FiftyOne Classificaton](https://docs.voxel51.com/api/fiftyone.core.labels.html#fiftyone.core.labels.Classification) to your dataset:

In [ ]:
body_system_labels = dataset.distinct("modality.label")

model.operation = "classify"

model.prompt = "As a medical expert your task is to classify this image into exactly one of the following types: " + ", ".join(body_system_labels)

dataset.apply_model(model, label_field="pred_modality")

Here's the ground truth:

In [4]:
dataset.first()['modality']

<Classification: {
    'id': '6830a40a7a3316c437168c05',
    'tags': [],
    'label': 'X-Ray',
    'confidence': None,
    'logits': None,
}>

And the model prediction:

In [5]:
dataset.first()['pred_modality.classifications']

[<Classification: {
     'id': '6830d550d69a8c1f151f7b90',
     'tags': [],
     'label': 'X-Ray',
     'confidence': None,
     'logits': None,
 }>]

# Using MedGemma for VQA

You can use the model for visual question answering as shown below. This example will use the same prompt on each [Sample](https://docs.voxel51.com/api/fiftyone.core.sample.html#module-fiftyone.core.sample) in the Dataset. Note the default system prompt:

In [6]:
print(model.system_prompt)

You are an expert radiologist, histopathologist, ophthalmologist, and dermatologist.

Your expert opinion is needed for in classifiying medical images. A given image may have multiple relevant classifications.  

You may be requested to select from a list of classifications, or asked to leverage your expertise medical for a diagnosis.

In any event, report your classifications as JSON array in this format: 

```json
{
    "classifications": [
        {
            "label": "descriptive medical condition or relevant label",
            "label": "descriptive medical condition or relevant label",
            ...,
        }
    ]
}
```

Always return your response as valid JSON wrapped in ```json blocks.  You may produce multiple lables if they are relevant or if you are asked to. Do not report your confidence.



You can customize the system prompt as follows to guide the model's response. Note that we are using an existing field on the sample by passing `prompt_field="question"` into the [`apply_model`](https://docs.voxel51.com/api/fiftyone.core.dataset.html) method of the [Dataset](https://docs.voxel51.com/api/fiftyone.core.dataset.html).

Note that if you want to parse the model output as a [FiftyOne Classification](https://docs.voxel51.com/api/fiftyone.core.labels.html#fiftyone.core.labels.Classification) then you need to very specifically prompt the model to output in a way that this integration expects, that is:


```json
{
    "classifications": [
        {
            "label": "your answer to the question",
            ...,
        }
    ]
}
```

Notice below:

In [ ]:
model.operation="classify"

model.system_prompt = """You have expert-level medical knowledge in radiology, histopathology, ophthalmology, and dermatology.

You will be asked a question and are required to provide your answer. Your answer must be in the following format:

```json
{
    "classifications": [
        {
            "label": "your answer to the question",
            ...,
        }
    ]
}
```

Always return your response as valid JSON wrapped in ```json blocks and respond only with one answer.
"""

dataset.apply_model(
    model, 
    label_field="pred_answer_6", 
    prompt_field="question_6"
    )

In [8]:
dataset.first()['question_6']

'Which organ is abnormal, heart or lung?'

In [9]:
dataset.first()['answer_6']

<Classification: {
    'id': '6830a40a7a3316c437168c0e',
    'tags': [],
    'label': 'Heart',
    'confidence': None,
    'logits': None,
}>

In [10]:
dataset.first()['pred_answer_6.classifications']

[<Classification: {
     'id': '6830d5a0d69a8c1f151f7bc2',
     'tags': [],
     'label': 'Heart',
     'confidence': None,
     'logits': None,
 }>]

For open-ended generation, you can use `vqa` mode. Note that with both modes you can use a single question on each sample as seen below:

In [ ]:
model.system_prompt = None # we need to clear the custom system prompt  

model.operation="vqa"

model.prompt = "Describe any anomolies in this the image that you observe."

dataset.apply_model(model, label_field="open_response")

In [12]:
print(dataset.first()["open_response"])

Okay, I will analyze the chest X-ray provided.

**Observations:**

*   **Cardiomegaly:** The heart appears enlarged (cardiomegaly). This is evident by the increased prominence of the cardiac silhouette, particularly the left ventricle.
*   **Mediastinal Devices:** There are multiple devices visible within the mediastinum, including a pacemaker/ICD device. The leads appear to be in standard positions.
*   **Pulmonary Vascularity:** The pulmonary vasculature appears relatively normal in terms of distribution and prominence.
*   **Lung Fields:** The lung fields appear clear with no obvious consolidation, effusions, or masses.
*   **Bones:** The bony structures of the ribs and sternum appear intact.
*   **Mediastinal Width:** The mediastinal width appears to be within normal limits.

**Potential Anomalies/Considerations:**

*   **Cardiomegaly:** The cardiomegaly could be due to various factors, including hypertension, valvular heart disease, cardiomyopathy, or congenital heart defects. Fur

Or use you can use a Sample field:

In [14]:
model.operation="vqa"

dataset.apply_model(
    model, 
    label_field="answer_5_vqa", 
    prompt_field="question_5"
    )

   0% ||------------------|  0/50 [3.2ms elapsed, ? remaining, ? samples/s] 

Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.


   2% |/------------------|  1/50 [6.7s elapsed, 5.5m remaining, 0.1 samples/s] 

Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.


   4% |-------------------|  2/50 [8.7s elapsed, 3.5m remaining, 0.2 samples/s] 

...

Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.


  18% |███/---------------|  9/50 [32.6s elapsed, 2.5m remaining, 0.3 samples/s] 

Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.


  20% |███----------------| 10/50 [33.9s elapsed, 2.2m remaining, 0.3 samples/s] 

Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.


In [ ]:
dataset.first()["question_5"]

In [ ]:
dataset.first()["answer_5"]

In [ ]:
dataset.first()["answer_5_vqa"]

# Evaluating the model

You can use FiftyOne's [Evaluation API](https://docs.voxel51.com/user_guide/evaluation.html) to evaluate model performance via the SDK. 

For example:

In [ ]:
results = dataset.evaluate_classifications(
    "pred_modality",
    gt_field="modality",
    eval_key="eval_simple",
)

You can then review as follows:

In [ ]:
results.print_report()

In [ ]:
plot = results.plot_confusion_matrix()
plot.show()

You can, of course, do all of this in the App as shown here: